In [38]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name)
)

Found GPU at: /device:GPU:0


In [39]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [40]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18425792267686007188, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2460657930699595835
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 549621189593322567
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648777152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17598710925453987371
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [41]:
#//data conversion
# bags of words




vocab = {}
word_encoding = 1 
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")
  bag = {}
  for word in words:
    if word in vocab:
      encoding = vocab[word]
    else:
      vocab[word] = word_encoding
      encoding = word_encoding 
      word_encoding+=1    

    if encoding in bag:
      bag[encoding] += 1

    else:
      bag[encoding] = 1

  return bag
  pass 










In [42]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1}
Negative: {1: 1, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 1, 13: 1, 10: 1, 11: 1, 12: 1, 9: 1}


In [43]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

text = "this is a test to see if this test will work is is test a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [44]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)
#maintain specific order

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


In [45]:
# movie imdb dataset
# 
%tensorflow_version 2.x 
from keras.datasets import imdb
from keras.preprocessing import sequence 
import keras 
import tensorflow as tf 
import os 
import numpy as np 
VOCAB_SIZE = 88584 
MAXLEN = 250 
BATCH_SIZE = 64 
(train_data,train_labels) , (test_data,test_labels) = imdb.load_data(num_words = VOCAB_SIZE) 



In [46]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [47]:
cd ../

/


In [48]:
ls


bin/                                       etc/    opt/    sys/
boot/                                      home/   proc/   tensorflow-1.15.2/
content/                                   lib/    root/   tmp/
datalab/                                   lib32/  run/    tools/
dev/                                       lib64/  sbin/   usr/
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media/  srv/    var/
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt/    swift/


In [49]:
train_data.shape

(25000, 250)

In [50]:
len(train_data[1])

250

In [51]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(VOCAB_SIZE,32),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1,activation="sigmoid")
])


In [52]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          2834688   
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [66]:
train_data.shape

(25000, 250)

In [53]:
model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ['acc'])

history = model.fit(train_data,train_labels,epochs = 30,validation_split=0.2)

Epoch 1/30
625/625 [==============================] - 15s 24ms/step - loss: 0.4188 - acc: 0.8054 - val_loss: 0.2926 - val_acc: 0.8788
Epoch 2/30
625/625 [==============================] - 14s 22ms/step - loss: 0.2389 - acc: 0.9084 - val_loss: 0.2914 - val_acc: 0.8848
Epoch 3/30
625/625 [==============================] - 14s 23ms/step - loss: 0.1812 - acc: 0.9341 - val_loss: 0.3189 - val_acc: 0.8886
Epoch 4/30
625/625 [==============================] - 14s 23ms/step - loss: 0.1489 - acc: 0.9466 - val_loss: 0.3511 - val_acc: 0.8880
Epoch 5/30
625/625 [==============================] - 14s 23ms/step - loss: 0.1282 - acc: 0.9559 - val_loss: 0.3580 - val_acc: 0.8754
Epoch 6/30
625/625 [==============================] - 14s 23ms/step - loss: 0.1123 - acc: 0.9627 - val_loss: 0.2959 - val_acc: 0.8870
Epoch 7/30
625/625 [==============================] - 14s 22ms/step - loss: 0.0954 - acc: 0.9683 - val_loss: 0.3315 - val_acc: 0.8834
Epoch 8/30
625/625 [==============================] - 14s 22ms

In [54]:
results = model.evaluate(test_data,test_labels)

782/782 [==============================] - 4s 6ms/step - loss: 0.9547 - acc: 0.8349


In [55]:
test_labels.shape

(25000,)

In [56]:
test_data.shape

(25000, 250)

In [57]:
print(results)

[0.9546781778335571, 0.834879994392395]


In [64]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  # print(tokens)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens ]
  # print(tokens)
  return sequence.pad_sequences([tokens],MAXLEN)[0]
text = "that was good movie was just amazing so amazing,nice awesome amazing"
endcoded = encode_text(text)
print(endcoded)




[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [68]:
# word_index

In [65]:
#reverse dict


reverse_word_index = {value: key for (key,value) in word_index.items() }
def decode_ed(integers):
  PAD = 0
  text = ""
  for num in integers:

    if num!=PAD:
      text+=reverse_word_index[num]+" "

      pass 
  return text[:-1]



print(decode_ed(endcoded))



that was good movie was just amazing so amazing nice awesome amazing


In [75]:
def predict(textitem):
  encoded_text = encode_text(textitem)
  # print(encoded_text.shape)
  pred_val = np.zeros((1,250))
  print(pred_val)

  pred_val[0] = encoded_text 
  result = model.predict(pred_val)
  print(result)
  pass


positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.9962159]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.